In [31]:
import pandas as pd
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  InputLayer
from tensorflow.keras.layers import  Dense
import tensorflow.keras.layers as layers
from sklearn.metrics import classification_report
import numpy as np

from sklearn.preprocessing import LabelEncoder

benign = pd.read_csv('C:\\Users\\gaibo\\OneDrive\\Escritorio\\Python Projects\\DNS Traffic Control\\Data\\Benign\\stateless_features-light_benign.pcap.csv')

malicious = pd.read_csv('C:\\Users\\gaibo\\OneDrive\\Escritorio\\Python Projects\\DNS Traffic Control\\Data\\Malicious\\stateless_features-light_compressed.pcap.csv')

data_merged = pd.concat([benign, malicious])

data_merged = data_merged.reset_index(drop=True)

# To avoid bias we are going to shuffle the dataset
data = data_merged.sample(frac=1).reset_index(drop=True)

from sklearn.model_selection import train_test_split

# Now lets split the dataset into training and testing data, we are going to use only the most important features for the model
X = data.drop(['class'], axis=1)
y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [32]:
#print the class distribution
print(Counter(data["class"]))

Counter({0: 60091, 1: 10241})


ValueError: could not convert string to float: 'kariya'

In [33]:

le = LabelEncoder()

X_train['longest_word'] = le.fit_transform(X_train['longest_word'].astype(str))

X_train['sld'] = le.fit_transform(X_train['sld'].astype(str))

X_test['longest_word'] = le.fit_transform(X_test['longest_word'].astype(str))

X_test['sld'] = le.fit_transform(X_test['sld'].astype(str))

X_train['longest_word'] = tensorflow.keras.utils.to_categorical(X_train['longest_word'], dtype='int64')
X_test['longest_word'] = tensorflow.keras.utils.to_categorical(X_test['longest_word'], dtype='int64')

X_train['sld'] = tensorflow.keras.utils.to_categorical(X_train['sld'], dtype='int64')
X_test['sld'] = tensorflow.keras.utils.to_categorical(X_test['sld'], dtype='int64')


# Lets normalize the numerical data
from sklearn.preprocessing import StandardScaler

# Create a scaler object
scaler = StandardScaler()

# Fit the data on the scaler object
X_train[['FQDN_count', 'subdomain_length', 'entropy', 'len', 'longest_word', 'sld']] = scaler.fit_transform(X_train[['FQDN_count', 'subdomain_length', 'entropy', 'len', 'longest_word', 'sld']])

X_test[['FQDN_count', 'subdomain_length', 'entropy', 'len', 'longest_word', 'sld']] = scaler.fit_transform(X_test[['FQDN_count', 'subdomain_length', 'entropy', 'len', 'longest_word', 'sld']])


#design the model
model = Sequential()
model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


#compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#train and evaluate the model
model.fit(X_train, y_train, epochs = 30, batch_size = 16, verbose = 1)


Epoch 1/30
3077/3077 [==============================] - 12s 1ms/step - loss: 0.3120 - accuracy: 0.8529
Epoch 2/30
3077/3077 [==============================] - 3s 1ms/step - loss: 0.2995 - accuracy: 0.8536
Epoch 3/30
3077/3077 [==============================] - 3s 1ms/step - loss: 0.2982 - accuracy: 0.8540
Epoch 4/30
3077/3077 [==============================] - 3s 1ms/step - loss: 0.2981 - accuracy: 0.8539
Epoch 5/30
3077/3077 [==============================] - 3s 1ms/step - loss: 0.2977 - accuracy: 0.8540
Epoch 6/30
3077/3077 [==============================] - 3s 1ms/step - loss: 0.2970 - accuracy: 0.8542
Epoch 7/30
3077/3077 [==============================] - 3s 1ms/step - loss: 0.2967 - accuracy: 0.8541
Epoch 8/30
3077/3077 [==============================] - 3s 1ms/step - loss: 0.2965 - accuracy: 0.8542
Epoch 9/30
3077/3077 [==============================] - 3s 1ms/step - loss: 0.2964 - accuracy: 0.8542
Epoch 10/30
3077/3077 [==============================] - 3s 1ms/step - loss: 0.29

In [47]:
import numpy as np
from sklearn.metrics import classification_report
from collections import Counter

# Get the predicted probabilities
y_estimate = model.predict(X_test)

y_estimate_class = np.zeros_like(y_estimate)
y_estimate_class[y_estimate >= 0.4] = 1  # Attack
y_estimate_class[y_estimate < 0.4] = 0  # Benign

# Print the first 5 predictions
print(y_estimate_class[:5])

# Print the number of 0 and 1 in y_estimate_class
print(Counter(y_estimate_class.flatten()))

# Print classification report
print(classification_report(y_test.tolist(), y_estimate_class.tolist()))

[[3.0768436e-01]
 [7.2627940e-05]
 [3.0768436e-01]
 [6.8801652e-05]
 [3.0768436e-01]]
Counter({0.30768436: 9988, 7.47067e-05: 179, 7.262794e-05: 143, 7.2766736e-05: 143, 4.2913012e-05: 141, 5.396586e-05: 129, 7.9633624e-05: 129, 6.540721e-05: 122, 5.4595694e-05: 118, 5.8917398e-05: 113, 3.4244706e-05: 111, 2.7900818e-05: 110, 5.9483384e-05: 106, 4.6783756e-05: 103, 4.2418036e-05: 101, 3.197521e-05: 98, 6.0177612e-05: 97, 6.0237096e-05: 94, 6.880165e-05: 89, 6.453339e-05: 84, 6.615454e-05: 83, 6.688706e-05: 79, 4.807414e-05: 74, 1.7717259e-05: 67, 8.17378e-05: 65, 1.7121578e-05: 63, 8.234267e-05: 63, 0.2563271: 63, 2.2665603e-05: 62, 3.3849712e-05: 61, 2.7578946e-05: 58, 7.3338924e-05: 56, 0.0004017949: 55, 5.9432296e-05: 53, 4.7519632e-05: 51, 2.5882686e-05: 51, 2.1534663e-05: 50, 7.527691e-05: 50, 0.25881982: 49, 3.9501778e-05: 48, 0.00040486455: 47, 5.495603e-05: 46, 0.0005593598: 44, 2.9825354e-05: 44, 0.00038433075: 43, 1.6059603e-05: 41, 0.0005183518: 40, 0.00019446015: 39, 0.0004

ValueError: Classification metrics can't handle a mix of binary and continuous targets